# CODI MODELS

In [1]:
import dask
import dask.dataframe as dd

In [2]:
from typing import Dict, Any
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix
import time
from sklearn.model_selection import train_test_split

class_metrics = {'accuracy_score':accuracy_score, 'f1_score':f1_score, 'precision_score':precision_score, 'recall_score':recall_score }
reg_metrics = {'mean_squared_error': mean_squared_error, 'r2_score':r2_score}

### OTRA PRUEBA

In [3]:
ddf_2 = dd.read_parquet("prepro/train/parquet-23.parq").compute()
ddf_3 = dd.read_parquet("prepro/train/parquet-114.parq").compute()
ddf_4 = dd.read_parquet("prepro/train/parquet-130.parq").compute()
ddf_5 = dd.read_parquet("prepro/train/parquet-35.parq").compute()
ddf_6 = dd.read_parquet("prepro/train/parquet-142.parq").compute()
ddf_7 = dd.read_parquet("prepro/train/parquet-31.parq").compute()
ddf_8 = dd.read_parquet("prepro/train/parquet-47.parq").compute()
ddf_9 = dd.read_parquet("prepro/train/parquet-39.parq").compute()
ddf_10 = dd.read_parquet("prepro/train/parquet-128.parq").compute()

ddf_total = dd.concat([ddf_2, ddf_3, ddf_4, ddf_5, ddf_6]).compute()

ddf_100 = dd.read_parquet("prepro/train/parquet-11.parq").compute()
ddf_101 = dd.read_parquet("prepro/train/parquet-59.parq").compute()
ddf_102 = dd.read_parquet("prepro/train/parquet-134.parq").compute()
ddf_103 = dd.read_parquet("prepro/train/parquet-91.parq").compute()

ddf_val = dd.concat([ddf_100, ddf_101, ddf_102, ddf_103]).compute()

ddf_total.drop(columns = ["buyer_d1", "buy_d7", "buyer_d14", "buyer_d28", 'iap_revenue_d7', "buy_d14", "buy_d28", "iap_revenue_d14", "iap_revenue_d28", "registration", "retention_d1_to_d7", "retention_d3_to_d7", "retention_d7_to_d14", "retention_d1", "retention_d3", "retentiond7"], axis=1, inplace = True)
ddf_val.drop(columns = ["buyer_d1", "buy_d7", "buyer_d14", "buyer_d28", 'iap_revenue_d7', "buy_d14", "buy_d28", "iap_revenue_d14", "iap_revenue_d28", "registration", "retention_d1_to_d7", "retention_d3_to_d7", "retention_d7_to_d14", "retention_d1", "retention_d3", "retentiond7"], axis=1, inplace = True)

y_train = ddf_total['buyer_d7']
X_train = ddf_total.drop(['buyer_d7', 'row_id'], axis = 1)

y_test = ddf_val['buyer_d7']
X_test = ddf_val.drop(['buyer_d7', 'row_id'], axis = 1)

In [4]:
from xgboost import XGBClassifier

# XGBClassifier Bernoulli for buy_d7

num_neg = sum(y_train == 0)
num_pos = sum(y_train == 1)

xg_class = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary:logistic",  # para clasificación binaria
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1,
    scale_pos_weight = (num_neg / num_pos)*0.95,
)

xg_class.fit(X_train,y_train)
preds = xg_class.predict(X_test)
for key, metr in class_metrics.items():
    print(key, metr(preds, y_test))

cm = confusion_matrix(y_test, preds)
print(cm)

accuracy_score 0.8054501885859813
f1_score 0.1562587379998136
precision_score 0.4514608859566447
recall_score 0.09447997971202345
[[293118  64272]
 [  8148   6706]]


### OTRA COSA

In [5]:
prediction = xg_class.predict(X_train)
#print(prediction.mean())
X_train["Classify"] = prediction
X_train2 = X_train[X_train["Classify"] == 1]
y_train2 = y_train[X_train["Classify"] == 1]

In [6]:
X_train2.drop(["Classify"], axis = 1, inplace=True)

/tmp/ipykernel_6944/905884129.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train2.drop(["Classify"], axis = 1, inplace=True)


In [7]:
from sklearn.metrics import mean_squared_log_error
reg_metrics2 = {"MSLE": mean_squared_log_error}

In [8]:
from xgboost import XGBRegressor

xg_reg_tweedie = XGBRegressor(
                  objective="reg:tweedie",
                  tweedie_variance_power=1.2,   # prueba entre 1.1 y 1.5
                  learning_rate=0.05,
                  max_depth=4,
                  n_estimators=800,
                  subsample=0.8,
                  colsample_bytree=0.8,
                  alpha=0,
                  reg_lambda=1,
              )

xg_reg_tweedie.fit(X_train2,y_train2)
preds = xg_reg_tweedie.predict(X_test)
for key, metr in reg_metrics2.items():
    print(key, metr(preds, y_test))

MSLE 0.01946183107793331


#

# SUBMISSION GENERATION

In [10]:
path_test = "prepro/test"

ddf_test = dd.read_parquet(
    path_test
).compute()

In [11]:
dff_test_feat = ddf_test.drop(['row_id'], axis = 1)

In [12]:
prediction = xg_class.predict(dff_test_feat)
#print(prediction.mean())
dff_test_feat["Classify"] = prediction
dff_test_feat2 = dff_test_feat[dff_test_feat["Classify"] == 1]
dff_test_feat2.drop(columns=["Classify"], inplace=True)

preds = xg_reg_tweedie.predict(dff_test_feat2)

/tmp/ipykernel_6944/628120029.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff_test_feat2.drop(columns=["Classify"], inplace=True)


In [13]:
import pandas as pd
iap_revenue_d7 = pd.Series(0, index=dff_test_feat.index)
iap_revenue_d7.loc[dff_test_feat["Classify"] == 1] = preds

/tmp/ipykernel_6944/1132212381.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.04857458 0.0792565  0.07593517 ... 0.12862262 0.05548548 0.11540903]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  iap_revenue_d7.loc[dff_test_feat["Classify"] == 1] = preds


In [14]:
df_result = pd.DataFrame({
    "row_id":ddf_test['row_id'],
    "iap_revenue_d7":iap_revenue_d7
})
df_result.to_csv("submission.csv", index=False)

## MAS PRUEBAS

In [ ]:
ddf_0 = dd.read_parquet("train_preprocess/parquet-2.parq").compute()
ddf_4 = dd.read_parquet("train_preprocess/parquet-140.parq").compute()
ddf_0.drop(columns = ["buy_d7", "buyer_d14", "buyer_d28", 'iap_revenue_d7', "buy_d14", "buy_d28", "iap_revenue_d14", "iap_revenue_d28", "registration", "retention_d1_to_d7", "retention_d3_to_d7", "retention_d7_to_d14", "retention_d1", "retention_d3", "retentiond7"], axis=1, inplace = True)
ddf_4.drop(columns = ["buy_d7", "buyer_d14", "buyer_d28", 'iap_revenue_d7', "buy_d14", "buy_d28", "iap_revenue_d14", "iap_revenue_d28", "registration", "retention_d1_to_d7", "retention_d3_to_d7", "retention_d7_to_d14", "retention_d1", "retention_d3", "retentiond7"], axis=1, inplace = True)

y_train = ddf_0[['buyer_d1', 'buyer_d7']]
X_train = ddf_0.drop(['buyer_d7', 'buyer_d1', 'row_id'], axis = 1)

y_test = ddf_4[['buyer_d1', 'buyer_d7']]
X_test = ddf_4.drop(['buyer_d7', 'buyer_d1', 'row_id'], axis = 1)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier

from sklearn import set_config 
set_config(enable_metadata_routing=True)

weights = [
    (y_train['buyer_d1'] == 0).sum() / (y_train['buyer_d1'] == 1).sum(),
    (y_train['buyer_d7'] == 0).sum() / (y_train['buyer_d7'] == 1).sum()
]

sample_weight = np.where(y_train == 1, weights, 1)

# num_neg = sum(y_train == 0)
# num_pos = sum(y_train == 1)

from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier

model = MultiOutputClassifier(
    XGBClassifier(
        objective="binary:logistic",
        learning_rate=0.05,
        max_depth=6,
        n_estimators=400,
        subsample=0.8,
        colsample_bytree=0.8
    )
)

model.fit(X_train, y_train, sample_weight=sample_weight)

UnsetMetadataPassedError: [sample_weight] are passed but are not explicitly set as requested or not requested for XGBClassifier.fit, which is used within MultiOutputClassifier.fit. Call `XGBClassifier.set_fit_request({metadata}=True/False)` for each metadata you want to request/ignore. See the Metadata Routing User guide <https://scikit-learn.org/stable/metadata_routing.html> for more information.

In [ ]:
Y_pred = model.predict(X_test)
Y_pred_bin = (Y_pred > 0.5).astype(int)
from sklearn.metrics import accuracy_score
subset_acc = accuracy_score(y_test, Y_pred_bin)
f1_micro = f1_score(y_test, Y_pred_bin, average="micro")
f1_macro = f1_score(y_test, Y_pred_bin, average="macro")
f1_weighted = f1_score(y_test, Y_pred_bin, average="weighted")

print(f1_micro, f1_macro, f1_weighted)

0.019734813444341658 0.018773077418826115 0.019709929330559272
